# SHAP explanations

In [ ]:
import trustyai

trustyai.init()

## Simple example

We start by defining our black-box model, typically represented by

$$
f(\mathbf{x}) = \mathbf{y}
$$

Where $\mathbf{x}=\{x_1, x_2, \dots,x_m\}$ and $\mathbf{y}=\{y_1, y_2, \dots,y_n\}$.

Our example toy model, in this case, takes an all-numerical input $\mathbf{x}$ and return a $\mathbf{y}$ of either `true` or `false` if the sum of the $\mathbf{x}$ components is within a threshold $\epsilon$ of a point $\mathbf{C}$, that is:

$$
f(\mathbf{x}, \epsilon, \mathbf{C})=\begin{cases}
\text{true},\qquad \text{if}\ \mathbf{C}-\epsilon<\sum_{i=1}^m x_i <\mathbf{C}+\epsilon \\
\text{false},\qquad \text{otherwise}
\end{cases}
$$

This model is provided in the `TestUtils` module. We instantiate with a $\mathbf{C}=500$ and $\epsilon=1.0$.

In [2]:
from trustyai.utils import TestUtils

center = 10.0
epsilon = 2.0

model = TestUtils.getSumThresholdModel(center, epsilon)

Next we need to define a **goal**.
If our model is $f(\mathbf{x'})=\mathbf{y'}$ we are then defining our $\mathbf{y'}$ and the counterfactual result will be the $\mathbf{x'}$ which satisfies $f(\mathbf{x'})=\mathbf{y'}$.

We will define our goal as `true`, that is, the sum is withing the vicinity of a (to be defined) point $\mathbf{C}$. The goal is a list of `Output` which take the following parameters

- The feature name
- The feature type
- The feature value (wrapped in `Value`)
- A confidence threshold, which we will leave at zero (no threshold)

In [3]:
from trustyai.model import output

decision = "inside"
goal = [output(name=decision, dtype="bool", value=True, score=0.0)]

We will now define our initial features, $\mathbf{x}$. Each feature can be instantiated by using `FeatureFactory` and in this case we want to use numerical features, so we'll use `FeatureFactory.newNumericalFeature`.

In [4]:
import random
from trustyai.model import feature

features = [feature(name=f"x{i+1}", dtype="number", value=random.random()*10.0) for i in range(3)]

As we can see, the sum of of the features will not be within $\epsilon$ (1.0) of $\mathbf{C}$ (500.0). As such the model prediction will be `false`:

In [5]:
feature_sum = 0.0
for f in features:
    value = f.value.as_number()
    print(f"Feature {f.name} has value {value}")
    feature_sum += value
print(f"\nFeatures sum is {feature_sum}")

Feature x1 has value 2.1516473114599046
Feature x2 has value 0.8137674993709809
Feature x3 has value 5.637541112355343

Features sum is 8.60295592318623


We execute the model on the generated input and collect the output

In [6]:
from org.kie.kogito.explainability.model import PredictionInput, PredictionOutput

goals = model.predictAsync([PredictionInput(features)]).get()

In [7]:
background = []
for i in range(10):
    _features = [feature(name=f"x{i+1}", dtype="number", value=random.random()*10.0) for i in range(3)]
    background.append(PredictionInput(_features))

We wrap these quantities in a `SimplePrediction`:

In [8]:
from trustyai.model import simple_prediction

prediction = simple_prediction(input_features=features, outputs=goals[0].outputs)

We can now instantiate the **explainer** itself.


In [9]:
from trustyai.explainers import SHAPExplainer

explainer = SHAPExplainer(background=background)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


We generate the **explanation** as a _dict : decision --> saliency_.


In [10]:
explanation = explainer.explain(prediction, model)

We inspect the saliency scores assigned by LIME to each feature

In [16]:
for saliency in explanation.getSaliencies():
    print(saliency)

Saliency{output=Output{value=true, type=boolean, score=-0.39704407681377063, name='inside'}, perFeatureImportance=[FeatureImportance{feature=Feature{name='x1', type=number, value=2.1516473114599046}, score=0.4, confidence= +/-0.39264863227014996}, FeatureImportance{feature=Feature{name='x2', type=number, value=0.8137674993709809}, score=0.35, confidence= +/-0.39264863227014996}, FeatureImportance{feature=Feature{name='x3', type=number, value=5.637541112355343}, score=0.15000000000000002, confidence= +/-0.5552890210036922}]}
